In [ ]:
#| hide
from xcube.utils import *

# xcube

> Extreme Multi-Label Text Classification

This file will become your README and also the index of your documentation.

## Install

Install using:

```sh
pip install xcube
```

or
```sh
conda install xcube
```

## How to use

Fill me in please! Don't forget code examples: